In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.misc import imread
import matplotlib.image as mpimg
import os

In [2]:
def load_all_labels(data_path):
    raw_labels = []
    with open(data_path) as input_labels:
        for line in input_labels:
            tokens = line.strip().split(',')
            raw_labels.append(tokens[1])
    return raw_labels

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
raw_labels = load_all_labels('data/train_v2.csv')
raw_labels = raw_labels[1:]
label_tf = CountVectorizer(binary=True, max_features=20)
labels = label_tf.fit_transform(raw_labels)

In [4]:
labels

<40479x17 sparse matrix of type '<class 'numpy.int64'>'
	with 116205 stored elements in Compressed Sparse Row format>

In [5]:
n_samples = labels.shape[0]
indices = np.random.permutation(n_samples)
#Number of images for train dataset and test dataset
n_train = 5000
n_test = 2000
# split the train and test 
#n_test = int(n_samples * 0.1)
print("Numbers of total images: ", n_samples)
print("Numbers of training images: ", n_train)
print("Number of test images: ", n_test)
#test = train_img[indices[:n_test]]
#train = train_img[indices[n_test:]]
test_labels = labels[indices[:n_test]]
train_labels = labels[indices[n_test:n_train+n_test]]

Numbers of total images:  40479
Numbers of training images:  5000
Number of test images:  2000


In [6]:
def load_image(data_path, indices, image_size):
    num_images = 0
    dataset = np.ndarray(shape=(len(indices), image_size, image_size, 3),dtype=np.float32)
    for i in tqdm(range(len(indices)), total = len(indices)):
        image = data_path + 'train_' + str(indices[i]) + '.jpg'
        image_data = mpimg.imread(image)[:,:,:3]
        #Normalization
        image_data = np.multiply(image_data, 1.0/255.0)
        dataset[num_images, :, :, :] = image_data
        num_images += 1
    print('Full dataset tensor:', dataset.shape)
    return dataset

In [7]:
data_path = "data/train-jpg/"
image_size = 256
print("Loading training images data...")
train = load_image(data_path, indices[n_test:n_train+n_test], image_size)
print("Loading testing images data...")
test = load_image(data_path, indices[:n_test], image_size)

  0%|          | 7/5000 [00:00<01:20, 62.22it/s]

Loading training images data...


  0%|          | 6/2000 [00:00<00:35, 56.12it/s]

Full dataset tensor: (5000, 256, 256, 3)
Loading testing images data...


100%|██████████| 2000/2000 [00:24<00:00, 81.15it/s]

Full dataset tensor: (2000, 256, 256, 3)


In [8]:
print(train.shape)
print(train_labels.shape)
print(test.shape)
print(test_labels.shape)

(5000, 256, 256, 3)
(5000, 17)
(2000, 256, 256, 3)
(2000, 17)


In [9]:
# Compute KL divergence
test_disb = np.sum(test_labels, axis=0)
train_disb = np.sum(train_labels, axis=0)
test_disb = test_disb / float(np.sum(test_disb))
train_disb = train_disb / float(np.sum(train_disb))

kl = np.sum(np.multiply(train_disb, (np.log(train_disb) - np.log(test_disb))))
print(kl)

kl = np.sum(np.multiply(test_disb, (np.log(test_disb) - np.log(train_disb))))
print(kl)

0.00125054630839
0.00130528539836


In [10]:
import os
import tensorflow as tf

#os.environ["CUDA_VISIBLE_DEVICES"]="3"

def get_session(gpu_fraction=0.1):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
    
import keras.backend as K
#K.set_session(get_session(1.0))

Using TensorFlow backend.


In [11]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.applications import vgg16
from keras.models import Model

img_width = 256
img_height = 256
batch_size = 32
num_classes = 17
epochs = 5
data_augmentation = False

x_train = train
x_test = test
y_train = train_labels.toarray()
y_test = test_labels.toarray()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (5000, 256, 256, 3)
5000 train samples
2000 test samples


In [41]:
#build the VGG16 network
base_model = vgg16.VGG16(weights = None, include_top = False, input_shape = (img_height, img_width, 3))
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [45]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation = 'sigmoid')(x)
model = Model(inputs = base_model.input, outputs = x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [46]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [47]:
def accuracy_with_threshold(y_true, y_pred):
    threshold = 0.5
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.equal(y_true, y_pred))

def hamming_dist(y_true, y_pred):
    threshold = 0.5
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.sum(K.abs(y_true - y_pred), axis=1))

In [48]:
def f_score(y_true, y_pred):
    threshold = 0.5
    y_pred = K.greater(y_pred, threshold)
    
    y_pred = tf.to_int32(y_pred)
    y_true = tf.to_int32(y_true)
    like = tf.ones_like(y_true)
    
    num_retrieves = tf.reduce_sum(y_pred, 1)
    
    num_relevants = tf.logical_and((tf.equal(y_true,like)),(tf.equal(y_pred,like)))
    num_relevants = tf.to_int32(num_relevants)                                
    num_relevants = tf.reduce_sum(num_relevants,1)
    
    total_relevants = tf.reduce_sum(y_true, 1)
    
    precision = tf.div(tf.to_float(num_relevants), tf.to_float(num_retrieves))
    recall = tf.div(tf.to_float(num_relevants), tf.to_float(total_relevants))
    beta = 2.
    beta_sqr = beta * beta
    #f1 = (1 + beta_sqr) * tf.div(tf.multiply(precision,recall),tf.add(beta_sqr * precision, recall))
    return tf.reduce_mean(recall)

In [49]:
stat = np.array(np.sum(y_train, axis = 0)).flatten()
print(stat)
scores = [np.sum(stat)/x for x in stat]
class_weights = {}
for i in range(len(scores)):
    class_weights[i] = scores[i] #if scores[i]>1.0 else 1.0
print(class_weights)

[1536   46  109   50    9 3518  271    8  524  464  333  878 4618 1032   44
   24  916]
{0: 9.3619791666666661, 1: 312.60869565217394, 2: 131.92660550458714, 3: 287.60000000000002, 4: 1597.7777777777778, 5: 4.0875497441728257, 6: 53.062730627306273, 7: 1797.5, 8: 27.442748091603054, 9: 30.991379310344829, 10: 43.183183183183182, 11: 16.378132118451024, 12: 3.1139021221307925, 13: 13.934108527131784, 14: 326.81818181818181, 15: 599.16666666666663, 16: 15.698689956331878}


In [50]:
sample_weight = np.zeros((y_train.shape[0], num_classes))

In [51]:
for i in range(num_classes):
    sample_weight[i] = scores.copy()

In [53]:
# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=[accuracy_with_threshold, hamming_dist, f_score], sample_weight_mode="temporal")


if not data_augmentation:
    print('Not using data augmentation.')
    hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                     validation_data=(x_test, y_test), sample_weight = sample_weight, shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Not using data augmentation.


ValueError: Found a sample_weight array for an input with shape (5000, 17). Timestep-wise sample weighting (use of sample_weight_mode="temporal") is restricted to outputs that are at least 3D, i.e. that have a time dimension.

In [17]:
print(hist.history)

{'hamming_dist': [1.6729333335876464, 1.3694666667938233, 1.1745333333969117, 1.0909333333969116, 1.0273333333333334, 0.97419999999999995, 0.92593333336512251, 0.87253333346048989, 0.84660000000000002, 0.81073333336512243, 0.77820000006357826, 0.75060000003178917, 0.72219999999999995, 0.6992666666666667, 0.67959999999999998, 0.64793333346048987, 0.61240000001589456, 0.5992000000476837, 0.56513333333333338, 0.54466666668256125, 0.52306666668256119, 0.49986666669845581, 0.4760000000158946, 0.45506666668256124, 0.4396000000635783, 0.4268666666984558, 0.41480000003178913, 0.4011333333412806, 0.3898000000158946, 0.38093333333333335, 0.36793333336512246, 0.36879999993642171, 0.35573333338101704, 0.35093333333730697, 0.33966666668256124, 0.34453333333333336, 0.33046666668256125, 0.3298666666706403, 0.328533333337307, 0.31953333333333334], 'loss': [0.25886614909172057, 0.19879011034965516, 0.17499071365992228, 0.16097121550242105, 0.15200471782286962, 0.14373897492090862, 0.13690229683717092, 

In [18]:
model.save('submission/submission2/planet2_4.h5')

In [19]:
pred = model.predict(x_test, batch_size=batch_size, verbose=0)
y_pred = (pred >= 0.5).astype(np.int32)
y_pred = y_pred.astype(np.int32)
y_test = y_test.astype(np.int32)
num_relevants = np.sum(y_test & y_pred, axis=1)
num_retrieves = np.sum(y_pred, axis=1)
total_relevants = np.sum(y_test, axis=1)

# precision
precision = np.divide(num_relevants.astype(np.float), num_retrieves.astype(np.float))
precision = np.nan_to_num(precision)
# recall
recall = np.divide(num_relevants.astype(np.float), total_relevants.astype(np.float))
recall = np.nan_to_num(recall)
beta = 2.
beta_sqr = beta * beta
f1 = (1 + beta_sqr) * np.divide((precision * recall),(beta_sqr * precision + recall))
f1 = np.nan_to_num(f1)
print('precision={}'.format(np.nanmean(precision)))
print('recall={}'.format(np.nanmean(recall)))
print('f1={}'.format(np.nanmean(f1)))

C:\yyh\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
C:\yyh\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


precision=0.8955761904761905
recall=0.8752970238095238
f1=0.8734377576686533


In [62]:
pre = model.predict(x_test,batch_size=32)

In [63]:
pre.shape

(1000, 17)

In [68]:
pre[0]

array([ 0.21239533,  0.00969336,  0.02384911,  0.05670618,  0.00668827,
        0.84235227,  0.01966204,  0.00752299,  0.19295782,  0.08035329,
        0.02582694,  0.1628222 ,  0.96848643,  0.14170928,  0.05123476,
        0.02263625,  0.18485051], dtype=float32)

In [73]:
y_pred = K.cast(K.greater(pre, 0.5), K.floatx())

In [74]:
y_pred

<tf.Tensor 'Cast_6:0' shape=(1000, 17) dtype=float32>

In [76]:
sess = tf.InteractiveSession()
y_pred = y_pred.eval()

In [78]:
y_pred[0]

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.], dtype=float32)

In [80]:
#for i in range(1000):
#    for j in range(17):
#        if pre[i][j] >=0.5:
#            pre[i][j] = 1
#        else:
#            pre[i][j]=0

In [82]:
accracy = K.mean(K.equal(y_test, y_pred)).eval()

In [83]:
accracy

0.92405879